# Age of Empires II Voobly Matches Data Exploration

## Load libraries and set options/environment

In [1]:
# System
import sys
import os
import datetime
from datetime import date, timedelta
sys.path.append('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/src/functions')

# Custom functions
from setup_sql_conn import setup_sql_conn,fetch_sql_data
from sql_read import sql_read

# Data Wrangling
import pandas as pd
import numpy as np
import re
from functools import reduce
from pandas_profiling import ProfileReport

# Visualization
import seaborn as sb
import matplotlib.pyplot as plt

# Set options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Read Data

In [2]:
db_conn,db_cursor,engine = setup_sql_conn('localhost','AOE2_VOOBLY')
match_data = sql_read('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/Input/match_data_query.sql',db_conn)

Time taken to read from server : 4.619019985198975


## Data Mining

In [3]:
match_data.head()

,ladder,Match_ID,Match_date,Match_time,Match_rating,Match_map,Match_length,Match_player_no,Match_mod,Player,New_Rating,Points,Team,Civilization,Victory,Military_Score,Economy_Score,Technology_Score,Society_Score,Total_Score,Units_Killed,Units_Lost,Buildings_Razed,Buildings_Lost,Units_Converted,Food,Wood,Stone,Gold,Trade_Profit,Tribute_Received,Tribute_Sent,Feudal_Time,Castle_Time,Imperial_Time,Map_Explored,Research_Count,Research_Percent,Total_Wonders,Total_Castles,Relics_Captured,Relic_Gold,Villager_High
0,Ladder: RM - 1v1,20850764,2019-11-02,20:46:00,1541,Black Forest,12,2,v1.5 RC | Longer Lasting Resou,yuriiarabchuk,1383,-4,2,CEL,0,0,301,1010,0,1311,0,2,0,0,0,1278,0,0,0,0,0,0,None,None,None,1.00,1,0.01,0,0,0,0,8
1,Ladder: RM - 1v1,20394127,2019-08-17,00:28:00,1252,Arabia,6,2,v1.5 RC | WololoKingdoms,[AML]MiguelBayona,1251,-17,2,GOT,0,0,295,190,0,485,0,1,0,0,0,374,280,0,0,0,0,0,None,None,None,0.18,1,0.00,0,0,0,0,14
2,Ladder: RM - 1v1,20394168,2019-08-17,00:35:00,1323,Arabia,9,2,v1.5 RC | WololoKingdoms,Kensema,1372,-22,2,MAG,0,0,411,280,0,691,0,2,0,0,0,1098,371,0,0,0,0,0,None,None,None,0.27,1,0.00,0,0,0,0,21
3,Ladder: RM - 1v1,20394169,2019-08-17,00:35:00,1554,Custom,9,2,v1.5 Beta R7,echoccccc,1574,16,1,CHI,1,0,463,310,0,773,0,0,0,0,0,1386,576,0,0,0,0,0,None,None,None,0.30,1,0.00,0,0,0,0,23
4,Ladder: RM - 1v1,20394169,2019-08-17,00:35:00,1554,Custom,9,2,v1.5 Beta R7,Nemo_Fulani,1534,-16,2,SAR,0,0,503,340,0,843,0,0,0,0,0,1289,540,0,0,0,0,0,None,None,None,0.34,0,0.00,0,0,0,0,23


In [15]:
match_data.dtypes

ladder          object
Match_ID         int64
Match_date      object
Match_time      object
Match_rating     int64
                 ...  
Food             int64
Wood             int64
Stone            int64
Gold             int64
Trade_Profit     int64
Length: 30, dtype: object